In [3]:
import praw
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime

In [192]:

# Authorized instance
def getAuthenticated(c_id, c_secret, userAgent, userName, Pswd):
    reddit = praw.Reddit(client_id= c_id,         # your client id
                                    client_secret= c_secret,      # your client secret
                                    user_agent= userAgent,        # your user agent
                                    username= userName,        # your reddit username
                                    password=Pswd
                                  )
    return reddit


In [193]:
#fayizaCre- Details1
reddit = getAuthenticated("eTNNwDnVsEukX4z0AHvDYQ", 
            "e9EKeODCP_lksBhiBxSw397bbgd9xw",   
            "HomelessDS",        
            "Faii50301",
            "cpsc@50301")

# choose the subreddit you want to retrieve data from
subreddit = reddit.subreddit("homeless")

The issue is likely due to the use of the last_id variable in the get_posts function, which is used to determine the next set of submissions to scrape. If the same last_id value is passed to the function each time it is called, it will result in the same set of submissions being retrieved repeatedly.

To fix this, you should update the last_id variable within the while loop so that it reflects the most recent submission retrieved from Reddit. This will allow you to continuously retrieve new submissions, rather than the same set of submissions repeatedly.

We have to check these:

Check the value of upto_timestamp, if it's not decreasing with each iteration then there is a problem with the code logic.

Verify that the get_posts function correctly returns the data frame with unique submissions. It could be that duplicates are being appended to all_posts_df each time.

Verify the logic for breaking the loop when the upto_timestamp is less than 1577836800, it could be that the break condition is not being satisfied.

In [97]:
# https://www.geeksforgeeks.org/how-to-convert-timestamp-string-to-datetime-object-in-python/
def dateANdTime(timestamp):
    dt_obj = datetime.fromtimestamp(timestamp)
    return dt_obj

In [225]:

#most important
def get_posts(n, last_id, collected_df):
    posts_list = [] 
    
    for submission in subreddit.controversial(limit=n):
        
        if (collected_df.empty == False):
            
            if(submission.id == collected_df['id'].iloc[0]): # if submission id has been collected then ignore this post
                print("Test: just collected:",submission.id ,
                      " = ", collected_df['id'].iloc[0] ,
                      " First id collected in previous batch \n" )
                break
        
        
        if(submission.author == None):   # if submission author is deleted then ignore this post
            continue

        info_list = []
        info_list.append(submission.id)   # Output: the submission's title
        info_list.append(submission.score)  # Output: the submission's score  
        info_list.append((str(submission.author)))  # output: the name of the author
        info_list.append(submission.num_comments) # output: the total number of comments made on this post
        info_list.append(submission.subreddit)  # Output: subreddit where this submission was posted
        info_list.append(submission.title)   # output: the title of the submission
        info_list.append(submission.selftext) # output: the body of the submission
        info_list.append(dateANdTime(submission.created_utc))  # output: the time stamp when this post was submitted

        posts_list.append(info_list)   # append all the above information into this posts_list

#     a = sorted(posts_list, key=lambda x: x[0], reverse = True)
    posts_df = pd.DataFrame(posts_list, columns = ['id', 'score', 'author', 'num_comments', 'subreddit', 'post_title' , 'post_Content', 'timestamp'])

    return posts_df

In [ ]:
# Epoch timestamp: 1577836800
# Timestamp in milliseconds: 1577836800000
# Date and time (GMT): Wednesday, January 1, 2020 12:00:00 AM



In [226]:
#most important

all_posts_df = pd.DataFrame(columns = ['id', 'score', 'author', 'num_comments',
                                       'subreddit', 'post_title' , 'post_Content', 'timestamp'])


limit = None
last_id = None
count = 0

while True:
    
    count +=1
    posts_df = get_posts(limit, last_id, all_posts_df) # get submission(post) - recursively
    all_posts_df = all_posts_df.append(posts_df)
    
    print("\ncount: " , count, "in While loop \n")
    print(posts_df)
    
    if len(posts_df) <= 0:
#     if upto_timestamp <= 1577836800: # getting posts upto this time Wednesday, January 1, 2020 12:00:00 AM
        print("\nreach end!!!")
        break
        
    last_id = posts_df['id'].iloc[-1]  # getting last post's id from dataframe
    upto_timestamp = posts_df.iloc[-1]['timestamp']
    

    print("\nlast_id: ", last_id)
    print("last_timestamp", upto_timestamp, "\n")
    
    
 
    

    
    
    #     #check statements
# #     ctime= datetime.now()
#      print("before: ", ctime.strftime("%H:%M:%S"))

C:\Users\faii_\AppData\Local\Temp\ipykernel_19888\3527990031.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_posts_df = all_posts_df.append(posts_df)



count:  1 in While loop 

          id  score                author  num_comments subreddit  \
0    106dh85      0            Gollumborn            33  homeless   
1     xpbzvf      0               treyj88            31  homeless   
2     p3zift      0              _iillii_            28  homeless   
3     wn4hzf      0         LALakersFan24            23  homeless   
4     xtxumd      0   Aggressive_Lion_587            19  homeless   
..       ...    ...                   ...           ...       ...   
875   ik773y      7      HauntedCrocodile             6  homeless   
876   gqqnak      8  AndrewEldritchHorror            22  homeless   
877   f7ovi5      5             myco-naut             6  homeless   
878   eyh3a6      6             zodiac707            32  homeless   
879   elj184      7             R0ntastic            10  homeless   

                                            post_title  \
0                                 Why all the garbage?   
1                       A fr

C:\Users\faii_\AppData\Local\Temp\ipykernel_19888\3527990031.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_posts_df = all_posts_df.append(posts_df)


In [166]:
def get_Unique_Values(repeating, df_columnName):
    unique_users_df = repeating.drop_duplicates(subset= df_columnName)
    return unique_users_df

In [168]:
newPost_nodes_df = get_Unique_Values(all_posts_df, 'author')
newPost_post_uniq = get_Unique_Values(all_posts_df, 'id')
print(len(newPost_nodes_df))
print(len(newPost_post_uniq))

646
933


In [174]:
hotPost_nodes_df = get_Unique_Values(all_posts_df, 'author')
hotPost_post_uniq = get_Unique_Values(all_posts_df, 'id')
print(len(hotPost_nodes_df))
print(len(hotPost_post_uniq))

496
717


In [198]:
topPost_post_uniq = get_Unique_Values(all_posts_df, 'id')
print(len(hotPost_post_uniq))

717


In [227]:
controversialPost_post_uniq = get_Unique_Values(all_posts_df, 'id')
print(len(controversialPost_post_uniq))

880


In [209]:
collection_df = pd.DataFrame(columns = ['id', 'score', 'author', 'num_comments',
                                       'subreddit', 'post_title' , 'post_Content', 'timestamp'])

collection_df = collection_df.append(newPost_post_uniq)
collection_df = collection_df.append(hotPost_post_uniq)
collection_df = collection_df.append(topPost_post_uniq)
collection_df = collection_df.append(controversialPost_post_uniq)



C:\Users\faii_\AppData\Local\Temp\ipykernel_19888\2533395087.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collection_df = collection_df.append(newPost_post_uniq)
C:\Users\faii_\AppData\Local\Temp\ipykernel_19888\2533395087.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collection_df = collection_df.append(hotPost_post_uniq)
C:\Users\faii_\AppData\Local\Temp\ipykernel_19888\2533395087.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collection_df = collection_df.append(topPost_post_uniq)
C:\Users\faii_\AppData\Local\Temp\ipykernel_19888\2533395087.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collection_df = collecti

In [1]:
post_check = get_Unique_Values(collection_df, 'id')
author_check = get_Unique_Values(collection_df, 'author')
print(len(post_check))
print(len(author_check))


NameError: name 'get_Unique_Values' is not defined

In [129]:
nodes_df.to_csv('.csv', index=False)

In [212]:
post_check.to_csv('newhotTopConPostsData.csv', index=False)

In [43]:
all_posts = pd.read_csv('newhotTopConPostsData.csv')


all_posts['timestamp'] = pd.to_datetime(all_posts['timestamp'])

# Split the 'timestamp' column into separate columns for year, month, day, hour, minute, and second
all_posts['year'], all_posts['month'], all_posts['day'], all_posts['hour'], all_posts['minute'], all_posts['second'] = all_posts['timestamp'].dt.year, all_posts['timestamp'].dt.month, all_posts['timestamp'].dt.day, all_posts['timestamp'].dt.hour, all_posts['timestamp'].dt.minute, all_posts['timestamp'].dt.second


In [48]:
all_posts = all_posts.sort_values(by=['year', 'month', 'day'], ascending=[False, False, False])


In [51]:
df_2019_later = all_posts.query('year > 2019')

df_before_2019 = all_posts.query('year <= 2019')


In [52]:
chunks = np.array_split(df_2019_later, 3)

for i, chunk in enumerate(chunks):
    chunk.to_csv(f'all_posts_2019_later{i}.csv', index=False)
    
df_before_2019.to_csv("all_posts_before_2019.csv", index=False)